In [2]:
! pip install keras_nlp

In [3]:
import keras_nlp
encoder1 = keras_nlp.layers.TransformerEncoder(intermediate_dim=120, num_heads=4)
encoder2 = keras_nlp.layers.TransformerEncoder(intermediate_dim=120, num_heads=4)
encoder3 = keras_nlp.layers.TransformerEncoder(intermediate_dim=120, num_heads=4)



In [4]:
import tensorflow as tf

def create_model_video_audio_then_text_with_transformers(encoder1):
    input_video = tf.keras.Input(shape=[768, 1])
    video_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_video)
    video_cnn = tf.keras.layers.MaxPooling1D()(video_cnn)
    video_cnn = encoder1(video_cnn)
    video_cnn = tf.keras.layers.Flatten()(video_cnn)

    input_audio = tf.keras.Input(shape=[768, 1])
    audio_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_audio)
    audio_cnn = tf.keras.layers.MaxPooling1D()(audio_cnn)
    audio_cnn = encoder1(audio_cnn)
    audio_cnn = tf.keras.layers.Flatten()(audio_cnn)

    concatenated_video_audio = tf.keras.layers.Concatenate(axis=1)([video_cnn, audio_cnn])

    input_text = tf.keras.Input(shape=[768, 1])
    text_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_text)
    text_cnn = tf.keras.layers.MaxPooling1D()(text_cnn)
    text_cnn = encoder1(text_cnn)
    text_cnn = tf.keras.layers.Flatten()(text_cnn)

    concatenated_all = tf.keras.layers.Concatenate(axis=1)([concatenated_video_audio, text_cnn])

    x = tf.keras.layers.Dense(200, activation='relu')(concatenated_all)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(90, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(56, activation='relu')(x)
    output = tf.keras.layers.Dense(2, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_video, input_audio, input_text], outputs=output)
    return model


model_video_audio_then_text = create_model_video_audio_then_text_with_transformers(encoder1)
model_video_audio_then_text.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 768, 1)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 768, 1)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 768, 32)        │            128 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 768, 32)        │            128 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 768, 1)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d             │ (None, 384, 32)        │              0 │ conv1d[0][0]           │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_1           │ (None, 384, 32)        │              0 │ conv1d_1[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 768, 32)        │            128 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, 384, 32)        │         12,184 │ max_pooling1d[0][0],   │
│ (TransformerEncoder)      │                        │                │ max_pooling1d_1[0][0], │
│                           │                        │                │ max_pooling1d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_2           │ (None, 384, 32)        │              0 │ conv1d_2[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 12288)          │              0 │ transformer_encoder[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 12288)          │              0 │ transformer_encoder[1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 24576)          │              0 │ flatten[0][0],         │
│                           │                        │                │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 12288)          │              0 │ transformer_encoder[2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 36864)          │              0 │ concatenate[0][0],     │
│ (Concatenate)             │                        │                │ flatten_2[0][0]        │
├──────────────────────

 Total params: 7,409,668 (28.27 MB)

 Trainable params: 7,409,268 (28.26 MB)

 Non-trainable params: 400 (1.56 KB)

In [5]:
def create_model_video_text_then_audio_with_transformers(encoder1):
    input_video = tf.keras.Input(shape=[768, 1])
    video_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_video)
    video_cnn = tf.keras.layers.MaxPooling1D()(video_cnn)
    video_cnn = encoder1(video_cnn)
    video_cnn = tf.keras.layers.Flatten()(video_cnn)

    input_text = tf.keras.Input(shape=[768, 1])
    text_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_text)
    text_cnn = tf.keras.layers.MaxPooling1D()(text_cnn)
    text_cnn = encoder1(text_cnn)
    text_cnn = tf.keras.layers.Flatten()(text_cnn)

    concatenated_video_text = tf.keras.layers.Concatenate(axis=1)([video_cnn, text_cnn])

    input_audio = tf.keras.Input(shape=[768, 1])
    audio_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_audio)
    audio_cnn = tf.keras.layers.MaxPooling1D()(audio_cnn)
    audio_cnn = encoder1(audio_cnn)
    audio_cnn = tf.keras.layers.Flatten()(audio_cnn)

    concatenated_all = tf.keras.layers.Concatenate(axis=1)([concatenated_video_text, audio_cnn])

    x = tf.keras.layers.Dense(200, activation='relu')(concatenated_all)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(90, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(56, activation='relu')(x)
    output = tf.keras.layers.Dense(2, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_video, input_text, input_audio], outputs=output)
    return model


model_video_text_then_audio = create_model_video_text_then_audio_with_transformers(encoder1)
model_video_text_then_audio.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 768, 1)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_4             │ (None, 768, 1)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 768, 32)        │            128 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_4 (Conv1D)         │ (None, 768, 32)        │            128 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 768, 1)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_3           │ (None, 384, 32)        │              0 │ conv1d_3[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_4           │ (None, 384, 32)        │              0 │ conv1d_4[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_5 (Conv1D)         │ (None, 768, 32)        │            128 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, 384, 32)        │         12,184 │ max_pooling1d_3[0][0], │
│ (TransformerEncoder)      │                        │                │ max_pooling1d_4[0][0], │
│                           │                        │                │ max_pooling1d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_5           │ (None, 384, 32)        │              0 │ conv1d_5[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_3 (Flatten)       │ (None, 12288)          │              0 │ transformer_encoder[3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_4 (Flatten)       │ (None, 12288)          │              0 │ transformer_encoder[4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 24576)          │              0 │ flatten_3[0][0],       │
│ (Concatenate)             │                        │                │ flatten_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_5 (Flatten)       │ (None, 12288)          │              0 │ transformer_encoder[5… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_3             │ (None, 36864)          │              0 │ concatenate_2[0][0],   │
│ (Concatenate)        

 Total params: 7,409,668 (28.27 MB)

 Trainable params: 7,409,268 (28.26 MB)

 Non-trainable params: 400 (1.56 KB)

In [6]:
def create_model_audio_text_then_video_with_transformers(encoder1):
    input_audio = tf.keras.Input(shape=[768, 1])
    audio_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_audio)
    audio_cnn = tf.keras.layers.MaxPooling1D()(audio_cnn)
    audio_cnn = encoder1(audio_cnn)
    audio_cnn = tf.keras.layers.Flatten()(audio_cnn)

    input_text = tf.keras.Input(shape=[768, 1])
    text_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_text)
    text_cnn = tf.keras.layers.MaxPooling1D()(text_cnn)
    text_cnn = encoder1(text_cnn)
    text_cnn = tf.keras.layers.Flatten()(text_cnn)

    concatenated_audio_text = tf.keras.layers.Concatenate(axis=1)([audio_cnn, text_cnn])

    input_video = tf.keras.Input(shape=[768, 1])
    video_cnn = tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same')(input_video)
    video_cnn = tf.keras.layers.MaxPooling1D()(video_cnn)
    video_cnn = encoder1(video_cnn)
    video_cnn = tf.keras.layers.Flatten()(video_cnn)

    concatenated_all = tf.keras.layers.Concatenate(axis=1)([concatenated_audio_text, video_cnn])

    x = tf.keras.layers.Dense(200, activation='relu')(concatenated_all)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(90, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(56, activation='relu')(x)
    output = tf.keras.layers.Dense(2, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_audio, input_text, input_video], outputs=output)
    return model


model_audio_text_then_video = create_model_audio_text_then_video_with_transformers(encoder1)
model_audio_text_then_video.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 768, 1)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_7             │ (None, 768, 1)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_6 (Conv1D)         │ (None, 768, 32)        │            128 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_7 (Conv1D)         │ (None, 768, 32)        │            128 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_8             │ (None, 768, 1)         │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_6           │ (None, 384, 32)        │              0 │ conv1d_6[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_7           │ (None, 384, 32)        │              0 │ conv1d_7[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_8 (Conv1D)         │ (None, 768, 32)        │            128 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, 384, 32)        │         12,184 │ max_pooling1d_6[0][0], │
│ (TransformerEncoder)      │                        │                │ max_pooling1d_7[0][0], │
│                           │                        │                │ max_pooling1d_8[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_8           │ (None, 384, 32)        │              0 │ conv1d_8[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_6 (Flatten)       │ (None, 12288)          │              0 │ transformer_encoder[6… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_7 (Flatten)       │ (None, 12288)          │              0 │ transformer_encoder[7… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, 24576)          │              0 │ flatten_6[0][0],       │
│ (Concatenate)             │                        │                │ flatten_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_8 (Flatten)       │ (None, 12288)          │              0 │ transformer_encoder[8… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_5             │ (None, 36864)          │              0 │ concatenate_4[0][0],   │
│ (Concatenate)        

 Total params: 7,409,668 (28.27 MB)

 Trainable params: 7,409,268 (28.26 MB)

 Non-trainable params: 400 (1.56 KB)